# Tree comparison

## case_insensitiv: changed GZMA_Canis_lupus to GZMA_Canis_lupus_1

In [6]:
import os
from ete3 import Tree as ete_tree
import dendropy 
from dendropy import Tree as dendropy_tree
from dendropy.calculate import treecompare

#from pycirclize import Circos
from io import StringIO
from Bio import Phylo
import csv
from pathlib import Path

# Parameter

In [7]:
converter = False

out_dir = str(os.getcwd())+"/out_activ_fct/sigmoid/"
#work_dir = str(os.getcwd())
input = "./inputs/"


multiple_comparison = True

reference_tree = '/Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/out_activ_fct/KLK_sigmoid_esm2_nj_manhattan_notraining.newick'#'/Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/inputs/KLK.newick' #str(out_dir) + "KLK.newick"
t2 = str(out_dir) + "KLK__esm2_nj_manhattan_tsne.newick" #"KLK_esm2_nj_euclidean_reference.newick" #"sigmoid/KLK_sigmoid_esm2_nj_euclidean_tsne"


# Comparison Metric

In [3]:
def get_comparison(t1, t2):
    
    print("Reference Tree: ",t1)
    print("Tree: ",t2+"\n")
    
    ### ETE
    tree1_ete = ete_tree(t1)
    tree2_ete = ete_tree(t2)

    rf, max_rf, common_leaves, parts_t1, parts_t2, discard_t1, discart_t2  = tree1_ete.robinson_foulds(tree2_ete,unrooted_trees=True)
    print("RF distance is %s over a total of %s" %(rf, max_rf))
    print("Partitions in tree2 that were not found in tree1:", len(parts_t1 - parts_t2))
    print("Partitions in tree1 that were not found in tree2:", len(parts_t2 - parts_t1))


    ### Dendropy
    tns = dendropy.TaxonNamespace()
    tree1 = dendropy_tree.get(path=t1,
            schema="newick",taxon_namespace=tns)#,case_sensitive_taxon_labels=True)
    tree2 = dendropy_tree.get(
            path=t2,
            schema="newick",taxon_namespace=tns)#,case_sensitive_taxon_labels=True)

    #!!!! Problem with reading case insensitive leaf nodes !!!!
    #changed GZMA_Canis_lupus to GZMA_Canis_lupus_1: case_insensitvity 

    tree1.encode_bipartitions()
    tree2.encode_bipartitions()

    print("euclidean_distance: "+str(treecompare.euclidean_distance(tree1, tree2)))
    print("symmetric_difference: "+str(treecompare.symmetric_difference(tree1, tree2)))
    print("false_positives_and_negatives: "+str(treecompare.false_positives_and_negatives(tree1, tree2)))
    print("robinson_foulds_distance: "+str(treecompare.robinson_foulds_distance(tree1, tree2)))


In [4]:
if multiple_comparison:

    pathlist = Path(out_dir).rglob('*.newick')
    for path in pathlist:
        path_in_str = str(path)
        print("\n")
        get_comparison(reference_tree, path_in_str)
else:
    get_comparison(reference_tree, t2)



Reference Tree:  /Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/inputs/KLK.newick
Tree:  /Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/out_activ_fct/sigmoid/KLK_sigmoid_esm2_upgma_cosine_1.newick

RF distance is 686 over a total of 850
Partitions in tree2 that were not found in tree1: 334
Partitions in tree1 that were not found in tree2: 352
euclidean_distance: 4.767913831236665
symmetric_difference: 690
false_positives_and_negatives: (353, 337)
robinson_foulds_distance: 110.28890614300006


Reference Tree:  /Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/inputs/KLK.newick
Tree:  /Users/adel.schmucklermann/Desktop/PyTorch_Lightning_VAE/out_activ_fct/sigmoid/KLK_sigmoid_esm2_upgma_ts_ss_1.newick

RF distance is 692 over a total of 850
Partitions in tree2 that were not found in tree1: 337
Partitions in tree1 that were not found in tree2: 355
euclidean_distance: 6.176589785308941
symmetric_difference: 696
false_positives_and_negatives: (356, 340)
robinson_f

# Draw Tree

In [8]:
from ete3 import Tree, TreeStyle

ts = TreeStyle()
ts.show_leaf_name = True
ts.scale =  120 # 120 pixels per branch length unit
ts.rotation = 90
ete_tree(reference_tree).show(tree_style=ts)

# Draw Tree with Circos

In [ ]:
tree_in = t2

#read color_file
dict_color = {}
with open(str(input) + "KLK_iTOL.txt") as csv_file:
    for row in csv.reader(csv_file, delimiter='\t'):
        if len(row) > 1:
            key = row[2]
            if key in dict_color:
                dict_color[key].append(row[0])
            else:
                dict_color[key] = [row[0]]
                
list_of_color = [(value,key) for key, value in dict_color.items()]                                    


#create tree
tree = Phylo.read(tree_in, "newick")

# Initialize circos sector with tree size
circos = Circos(sectors={"Tree": tree.count_terminals()})
sector = circos.sectors[0]

# Plot tree
track = sector.add_track((50, 100))
track.tree(tree,leaf_label_size=4, node_color_list=list_of_color)

fig = circos.plotfig()
#circos.savefig(str(out_dir) + "KLK_prott5.png", dpi=300)

# Format change

## Convert h5 to npz

In [ ]:
if converter: 

    h5_file = str(input) + "KLK_prott5.h5"
    prott5 = True

    #make numpy file with (seq header, embedding)
    esm2 = h5py.File(h5_file, 'r')
    list(esm2.keys())
    # List all groups
    keys = list(esm2.keys())
    #print(keys) 
    # Get the data
    esm2values = []
    headers = []
    for key in keys:
        #data
        emb = esm2[key][:]
        #value = np.array(esm2[key][:])
        #print(value)
        if prott5:
            esm2values.append(emb[0])
        else:    
            esm2values.append(emb)
        #headers
        key = key.replace("isoform=", "isoform_").replace("=", "").replace(":", "_")

        #check case-sensitivity
        if key.lower() in (item.lower() for item in headers):
            print('The string is in list: '+key)
            key = key+"_01"
            print('New key: '+key)

        headers.append(key)

    #np.savez("/Users/adel.schmucklermann/Desktop/FoPra/KLK/KLK_prott5.npz", headers=headers, embedding=esm2values)

## Convert Nexus to Newick

In [ ]:
if converter:
    import dendropy
    post_trees = dendropy.TreeList()
    post_trees.read(
             file=open("/Users/adel.schmucklermann/Downloads/KLK.nexus", "r"),# str(out_dir) + "KLK.nexus"
             schema="nexus")
    post_trees.write(
         path=str(out_dir) + "KLK.newick",schema="newick")

    #changed GZMA_Canis_lupus to GZMA_Canis_lupus_1: case_insensitvity 